# Section 1. Introduction

An **interpreter system** is a type of program that reads and executes instructions directly, translating them line-by-line into actions without first compiling them. It’s like a translator that interprets human-like commands and immediately performs the corresponding behavior.

In this project, we designed a **Snake Game Interpreter** — a simple rule-based interpreter that allows the snake to move across a grid, eat fruits, and reach a goal point using a set of defined opcodes.

#### Why a Snake Interpreter?
We selected this project because it combines **logic execution** with **visual behavior**, allowing us to demonstrate lexical analysis, parsing, and command execution in an engaging and understandable way. It also mimics how simple game engines interpret player inputs, which is useful for AI game logic and command-based automation.

#### Target Task
The interpreter reads snake movement commands written in our custom scripting language, processes them, executes the movement step-by-step, and displays the updated grid state after each command. The snake must:
- Move in four directions (up, down, left, right)
- Eat fruits placed on the grid
- Avoid colliding with walls or itself
- Optionally, use loop instructions for repeated patterns


# Section 2. Input Language Description

Our custom Snake Script consists of three main opcodes:

| Opcode | Syntax | Description |
|--------|---------|-------------|
| `MOVE` | MOVE [direction] [steps] | Moves the snake in the given direction for the given number of steps. |
| `EAT` | EAT | Eats the fruit if the snake's head is currently on the fruit. |
| `LOOP` | LOOP [start_line] [end_line] [count] | Repeats a block of code between two line numbers for the given count. |

### Tokens:
- `MOVE` keyword
- `EAT` keyword
- `LOOP` keyword
- Direction literals: `UP`, `DOWN`, `LEFT`, `RIGHT`
- Integer literals: positive integers for steps or loop counts
- Special symbols: newline (`\n`) separates commands

### Grammar Rules:
<command> ::= MOVE <direction> <int> | EAT | LOOP <start_line> <end_line> <int>
<direction> ::= UP | DOWN | LEFT | RIGHT
<int> ::= [1-9][0-9]*


### Examples:
**Valid Input:**
MOVE RIGHT 3
MOVE UP 2
EAT
MOVE LEFT 1

**Invalid Input:**
MOVE SIDEWAYS 2 # invalid direction
MOV RIGHT 3 # typo in opcode
EAT NOW # unexpected argument

When invalid input is detected, the interpreter raises a descriptive error message (e.g., “Invalid direction: SIDEWAYS”).

# Section 3. System Design

### Libraries Used:
**Built-in:**
- `re` – for lexical tokenization
- `sys` – for exit and error handling
- `copy` – for safely duplicating grid state

**Third-party:**
- None (pure Python)

### Architecture Overview
1. **Lexer** – breaks input commands into tokens.
2. **Parser** – validates token sequences based on syntax rules.
3. **Executor** – updates the grid and snake position step-by-step.

### Data Flow:
User Input → Lexer → Parser → Executor → Grid Output

### Error Handling:
- **Syntax Errors:** Invalid opcodes or direction.
- **Runtime Errors:** Moving into walls or self-collision.
- **Logic Errors:** Attempting to `EAT` when no fruit is present.

# Section 4. Data Preprocessing and Cleaning

- Describe the overall architecture of your interpreter. In this section, you must include the following:  
  - An overview of the three main components: **Lexer (Tokenizer)**, **Parser**, and **Executor (Interpreter Engine)**.  
  - A diagram or structured explanation of how data flows from **input → tokenization → parsing → execution → output**.  
  - Details about error handling strategies (e.g., syntax errors, runtime errors, invalid inputs).  
  - Justification for your design decisions. Why did you choose a particular parsing method? Why did you structure the interpreter this way?

In [1]:
import re
import sys
import copy

# Define constants
DIRECTIONS = {"UP": (-1, 0), "DOWN": (1, 0), "LEFT": (0, -1), "RIGHT": (0, 1)}

# Lexer: Tokenizes input lines
def lexer(line):
    tokens = re.findall(r"[A-Z]+|\d+", line.upper())
    return tokens

# Section 5. Implementation Details

- Provide and explain the implementation of your interpreter step by step. Show the source code for each component:  
  - **Lexer**: how tokens are identified and categorized.  
  - **Parser**: how the structure of the commands is validated.  
  - **Executor**: how the commands are executed.  
  - **Error handling**: how the system responds to invalid inputs.  
- Each code block should be accompanied by an explanation.

In [2]:
# 5.1 Lexer
def tokenize_script(script):
    lines = script.strip().split('\n')
    tokenized = []
    for i, line in enumerate(lines, start=1):
        if not line.strip():
            continue
        tokens = lexer(line)
        tokenized.append((i, tokens))
    return tokenized

# Example
script = """MOVE RIGHT 3
MOVE UP 2
EAT
"""
tokenize_script(script)

[(1, ['MOVE', 'RIGHT', '3']), (2, ['MOVE', 'UP', '2']), (3, ['EAT'])]

In [3]:
# 5.2 Parser
def parse(tokens):
    parsed = []
    for line_no, parts in tokens:
        if parts[0] == "MOVE":
            if len(parts) != 3 or parts[1] not in DIRECTIONS:
                raise SyntaxError(f"Invalid MOVE syntax at line {line_no}")
            parsed.append(("MOVE", parts[1], int(parts[2])))
        elif parts[0] == "EAT":
            if len(parts) != 1:
                raise SyntaxError(f"Invalid EAT syntax at line {line_no}")
            parsed.append(("EAT",))
        elif parts[0] == "LOOP":
            if len(parts) != 4:
                raise SyntaxError(f"Invalid LOOP syntax at line {line_no}")
            parsed.append(("LOOP", int(parts[1]), int(parts[2]), int(parts[3])))
        else:
            raise SyntaxError(f"Unknown opcode '{parts[0]}' at line {line_no}")
    return parsed

In [4]:
# 5.3 Executor
class SnakeGame:
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.grid = [['.' for _ in range(width)] for _ in range(height)]
        self.snake = [(height // 2, width // 2)]
        self.fruit = (1, 1)
        self.grid[self.fruit[0]][self.fruit[1]] = 'F'
        self.update_grid()

    def update_grid(self):
        for i in range(self.height):
            for j in range(self.width):
                if (i, j) == self.fruit:
                    self.grid[i][j] = 'F'
                elif (i, j) in self.snake:
                    self.grid[i][j] = 'S'
                else:
                    self.grid[i][j] = '.'

    def display(self):
        for row in self.grid:
            print(' '.join(row))
        print()

    def move(self, direction, steps):
        dx, dy = DIRECTIONS[direction]
        for _ in range(steps):
            head_x, head_y = self.snake[0]
            new_head = (head_x + dx, head_y + dy)
            if not (0 <= new_head[0] < self.height and 0 <= new_head[1] < self.width):
                raise RuntimeError("Snake hit the wall!")
            if new_head in self.snake:
                raise RuntimeError("Snake bit itself!")
            self.snake.insert(0, new_head)
            self.snake.pop()
            self.update_grid()
            self.display()

    def eat(self):
        if self.snake[0] == self.fruit:
            self.snake.append(self.snake[-1])  # grow
            print("Fruit eaten! Snake grew.")
            self.fruit = (self.height - 2, self.width - 2)
            self.update_grid()
            self.display()
        else:
            raise RuntimeError("No fruit to eat here!")

In [5]:
# 5.4 Interpreter
def run_interpreter(script):
    tokens = tokenize_script(script)
    parsed = parse(tokens)
    game = SnakeGame()

    i = 0
    while i < len(parsed):
        cmd = parsed[i]
        if cmd[0] == "MOVE":
            game.move(cmd[1], cmd[2])
        elif cmd[0] == "EAT":
            game.eat()
        elif cmd[0] == "LOOP":
            start, end, count = cmd[1], cmd[2], cmd[3]
            for _ in range(count):
                for j in range(start - 1, end):
                    inner_cmd = parsed[j]
                    if inner_cmd[0] == "MOVE":
                        game.move(inner_cmd[1], inner_cmd[2])
                    elif inner_cmd[0] == "EAT":
                        game.eat()
        i += 1

# Section 6. Testing with Valid and Invalid Inputs

- Demonstrate how your interpreter works by running a variety of test cases.  
- Show valid commands and their outputs.  
- Show invalid commands and the corresponding error messages.  
- Discuss how these test cases prove the correctness and robustness of your interpreter.

In [6]:
script = """
MOVE RIGHT 3
MOVE UP 2
EAT
MOVE LEFT 1
"""
run_interpreter(script)

. . . . . . . . . .
. F . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . S . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

. . . . . . . . . .
. F . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . S . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

. . . . . . . . . .
. F . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . S .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

. . . . . . . . . .
. F . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . S .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

. . . . . . . . . .
. F . . . . . . . .
. . . . . . . . . .
. . . . . . . . S .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . 

RuntimeError: No fruit to eat here!

In [7]:
script = """
MOVE SIDEWAYS 2
"""
try:
    run_interpreter(script)
except Exception as e:
    print("Error:", e)


Error: Invalid MOVE syntax at line 1


# Section 7. Extensions and Additional Features

We added the following beyond the basic requirements:
1. **Loop opcode** (`LOOP start end count`) — allows repeating a block of commands.
2. **Dynamic fruit relocation** — after eating, the fruit moves to another position.
3. **Wall and self-collision detection** — prevents illegal moves.
4. **Visual grid output** — displays the game state step-by-step.

# Section 8. Insights and Conclusions

Through this project, we learned how interpreters transform human-like commands into executable actions.  
We explored the core phases: **tokenization, parsing, execution, and error handling**, and saw how they connect in real-world applications like scripting and game automation.

**Strengths:**
- Clear opcode design
- Strong error handling
- Interactive and visual feedback

**Limitations:**
- Simple grid (no GUI)
- Fixed fruit positions (can be improved with randomness)

**Future Improvements:**
- Add variable handling and conditional logic
- Allow user-defined fruit locations
- Expand into GUI using `tkinter` or `pygame`


# Section 9. References

- Cite relevant references that you used in your project. All references must be cited, including:  
  - **Scholarly Articles**  
    - Cite in APA format, and put a description of how you used it for your work.  
  - **Online references, blogs, articles that helped you come up with your project**  
    - Put the website, blog, or article title, link, and how you incorporated it into your work.  
  - **Artificial Intelligence (AI) Tools**  
    - Put the model used (e.g., ChatGPT, Gemini), the complete transcript of your conversations with the model (including your prompts and its responses), and a description of how you used it for your work.

# Final Project Presentation

- Here are some guidelines regarding the final project presentation:  
  - Each group is given **20 minutes**: 15 minutes to present, and 5 minutes for Q&A.  
  - Presentations will be done **face-to-face** (sign-up for the schedule will be shared later).  
  - Open all the necessary files before your allotted presentation time slot. Do not wait until the presentation itself to load anything.  
  - All members should be present and should discuss a part in the final project presentation.